# Compression Lab

Due Tuesday, April 5 at 8 AM

## Introduction

The goal of compression is to represent a file using fewer bits. Compression was obviously very important many years ago when hard drive storage was measured in megabytes, not gigabytes or terabytes. But it remains relevant today in the age of small personal devices and big data. If the music on your phone were stored as uncompressed WAV files, you wouldn't be able to fit much music; fortunately, MP3 files are often about 10 times smaller. At the other extreme, even data centers that can store petabytes of data are struggling to keep up with satellites that are beaming terabytes of data back to Earth every second. Compression remains a relevant problem in the modern world.

There are two kinds of compression: lossy and lossless. Lossy compression is straightforward; to achieve a smaller file, we throw away some (hopefully less important) data. For example, MP3 leverages research on human psychoacoustics to discard frequencies that we wouldn't be able to hear anyway. On the other hand, lossless compression seems too good to be true. How can it be possible to achieve a smaller file without losing anything at all? We explore this question in this lab.

## Huffman Coding

In a uniform coding scheme, we use an equal number of bits to represent 
every symbol. For example, if there were 5 symbols in our world, a uniform
coding scheme might use a 3-bit code for each symbol:

A | B | C | D | E 
:--:|:--:|:--:|:--:|:--:
000 | 001 | 010 | 011 | 100

A table like this one that maps symbols to their codes is called a *codebook*.

But not all symbols are equally common. Suppose the actual probabilities 
of each letter were

A | B | C | D | E 
:--:|:--:|:--:|:--:|:--:
.05 | .15 | .20 | .25 | .35

If we used a shorter code to represent E, maybe we could have a shorter 
text on average. Huffman coding does exactly this. It constructs a code for 
each symbol using the following algorithm:

1. Find the two symbols with the smallest probabilities. Add a 1 to 
the beginning of the code for one symbol and a 0 to 
the beginning of the code for the other. (So far, the code for A is "1" and 
the code for B is "0".)
2. "Merge" those two symbols, and treat this merged symbol as a 
single symbol whose probability is the sum of the probabilities of the two 
symbols you merged. Repeat Steps 
1 and 2. (We now have a symbol AB whose probability is $.05 + .15 = .2$.
The two symbols now with the smallest probabilities are now 
AB and C. So we prepend a 1 to the beginning of the codes for A and B 
and a 0 to the beginning of the code for C. So the code for A becomes "11", 
the code for B becomes "10", and the code for C becomes "0".)
3. Repeat until all the symbols have been merged together.

To check your understanding of Huffman coding, follow the instructions 
above and make sure you get a codebook that looks like:

A | B | C | D | E 
:--:|:--:|:--:|:--:|:--:
111 | 110 | 10 | 01 | 00

(You might get something slightly different depending on which symbol you 
assigned a 1 to and which symbol you assigned a 0 to, at each step. As long as 
your codes are all of the same length, you should be okay.)

Let $L$ represent the length of the code for a randomly chosen symbol. Under 
uniform coding, $L=3$. Under Huffman coding:
$$ \textrm{E}[L] = .05(3) + .15(3) + .2(2) + .25(2) + .35(2) = 2.4. $$

In this lab, you will implement a simple Huffman coder. The first function has been implemented for you. You will have to implement three others.

In [1]:
def get_character_counts(text):
    """Count the number of occurrences of each symbol in a text.                                  
                                                                                                  
    Arguments:                                                                                    
    text -- string of text                                                                        
                                                                                                  
    Output:                                                                                       
    a dict that maps symbols to the number of times they appear                                   
      in the text                                                                                 
    """
    counts = {}
    # iterate over the characters in our text                                                     
    for char in text:
        # if this character is not already in our dict, add it                                    
        # (with a count of 1)                                                                     
        if char not in counts:
            counts[char] = 1
        # otherwise, add 1 to its count                                                           
        else:
            counts[char] += 1
    return counts

In [2]:
def generate_codebook(character_counts):
    """Uses Huffman coding to generate a codebook from character 
    counts / probabilities.

    Arguments:
    character_counts -- a dict that maps each character to either 
      its count or its probability.
                                                                                                  
    Output:                                                                                       
    a dict that maps symbols to a code consisting of 0's and 1's                                      
    """

    # initialize codebook with the same characters as character_counts                                   
    codebook = {}
    for symbol in character_counts.keys():
        codebook[symbol] = ""

    # YOUR CODE HERE
      
    # use this method in conjunction with a sort(reverse = False, key = sort_func) on a list in order to
    # sort a list of tuples based on values, in ascending order
    def sort_func(list_of_tuples):
        return list_of_tuples[1]
    
    #combines the key and values of the two tuples and return it
    def merge_tuple(tuple1,tuple2):
        temp_new_tuple = tuple1 + tuple2
        #tuple now looks like: (key1,value1,key2,value2)
        new_key = temp_new_tuple[0] + temp_new_tuple[2]
        new_value = temp_new_tuple[1] + temp_new_tuple[3]
        
        new_tuple = (new_key,new_value)
        
        return new_tuple
    
    def add_code_to_codebook(tuple1,tuple2):
        temp_new_tuple = tuple1 + tuple2
        #tuple now looks like: (key1,value1,key2,value2)
        key1 = temp_new_tuple[0]
        key2 = temp_new_tuple[2]
        
        for char in key1:
            codebook[char] = "1" + codebook[char]
            
        for char in key2:
            codebook[char] = "0" + codebook[char]
    
    """
    character_counts_list_form = list(character_counts.items() )
    # returns a list form of tuples in character_counts
    #print(character_counts_list_form[:10])
    
    #sort the list of tuples in character_counts dict based on values in ascending order
    character_counts_list_form.sort(reverse=False, key = sort_func)
    #print(character_counts_list_form[:10])
    #print(len(character_counts_list_form))
    
    smallest_value = character_counts_list_form[0]
    second_smallest_value = character_counts_list_form[1]
    #print(smallest_value)
    #print(second_smallest_value)
    
    #Before merging tuple: using smallest two tuples[value],  '1' + codebook[key], using the first key contents and do
    # '0' + codebook[key] for the second key contents
    add_code_to_codebook(smallest_value, second_smallest_value)
    
    for key,value in codebook.items():
        
        print(key + " " + value)
    
    
    #using smallest two values, remove first two tuples in list of character_counts
    character_counts_list_form.remove(smallest_value)
    character_counts_list_form.remove(second_smallest_value)
    print(character_counts_list_form[:10])
    
    merged_tuple = merge_tuple(smallest_value,second_smallest_value)
    print(merged_tuple)
    
    #prepend merged tuple to beginning to sorted list of tuples after removal:
    #note: it's not just a direct insert! :) Everything else gets pushed "out" of the way
    character_counts_list_form.insert(0, merged_tuple)
    print(character_counts_list_form[:10])
    
    """
    
    
    character_counts_list_form = list(character_counts.items() )
    # returns a list form of tuples in character_counts
     #print(character_counts_list_form[:10])
    #character_counts_list_form.sort(reverse=False, key = sort_func)
    #print(character_counts_list_form)
    
    while len(character_counts_list_form) > 1:
    
        #print(len(character_counts_list_form))
        
        
     
        #sort the list of tuples in character_counts dict based on values in ascending order
        character_counts_list_form.sort(reverse=False, key = sort_func)
        #print(character_counts_list_form[:10])
        #print(len(character_counts_list_form))
        
        smallest_value = character_counts_list_form[0]
        second_smallest_value = character_counts_list_form[1]
        #print(smallest_value)
        #print(second_smallest_value)
        
        #using smallest two values, remove first two tuples in list of character_counts
        character_counts_list_form.remove(smallest_value)
        character_counts_list_form.remove(second_smallest_value)
        #print(smallest_value)
        #print(second_smallest_value)
        
        #Before merging tuple: using smallest two tuples[value],  '1' + codebook[key], using the first key contents and do
        # '0' + codebook[key] for the second key contents
        add_code_to_codebook(smallest_value, second_smallest_value)
        
        #for key,value in codebook.items():
        
            #print(key + " " + value)
       
       
    
        merged_tuple = merge_tuple(smallest_value,second_smallest_value)
        #print(merged_tuple)
    
        #prepend merged tuple to beginning to sorted list of tuples after removal:
        #note: it's not just a direct insert! :) Everything else gets pushed "out" of the way
        character_counts_list_form.insert(0, merged_tuple)
        #print(character_counts_list_form[:10])
     
       
    

    return codebook

In [3]:
def encode(text, codebook):
    """Encodes a text using codebook.                                                             
                                                                                                  
    Arguments:                                                                                    
    text -- string of text to encode                                                              
    codebook -- dict that maps each symbol to a string of 0's and 1's                             
                                                                                                  
    Output:                                                                                       
    a string of 0's and 1's representing the encoded text                                         
    """
    encoded_text = ""

    # YOUR CODE HERE
    #raise NotImplementedError()
    """
    for key,value in codebook.items():
        print(key + " " + value) 
        """
    for char in text:
        
        encoded_text += codebook[char]
            

    return encoded_text

In [4]:
def decode(encoded_text, codebook):
    """Decodes encoded text using codebook.                                                       
                                                                                                  
    Arguments:                                                                                    
    encoded_text -- a string of 0's and 1's representing the encoded text                         
    codebook -- dict that maps each symbol to a code string of 0's and 1's                        
                                                                                                  
    Output:                                                                                       
    a string representing the decoded text                                                        
    """
    decoded_text = ""

    # reverse the codebook so that it maps code strings back to symbols                           
    reverse_codebook = {val: key for key, val in codebook.items()}

    # YOUR CODE HERE
    #raise NotImplementedError()
    substring = ""
    
    for char in encoded_text:
        
        substring += char
        
        if substring in reverse_codebook.keys():
            
            decoded_text += reverse_codebook[substring]
            substring = ""
            
            
#     """
#     #NOTE: Code below is REALLY inefficient. Technically, it worked and I was able to
#     #decode most of the encoded text successfully (out of 1000) , but it wasn't feasible to wait until 
#     #program finished. Got help to write far more efficient code above. [proof: print(decoded_text)
#     in while loop]
    
#    """
#      """
    
#     substring_index = 1
    
#     while len(encoded_text) > 0:
        
        

        
#         # get one character at a time from encoded_text to try match with reversed codebook
#         chosen_substring = encoded_text[:substring_index]
        
        
        
#         if(chosen_substring in reverse_codebook.keys()):
                
#             decoded_text += reverse_codebook[chosen_substring]
#             #print(len(decoded_text))
#             #print(decoded_text)
            
#             #delete part of encoded_text that has been used to assign a symbol to:
#             encoded_text = encoded_text[substring_index:]
#             #print(len(encoded_text))
                
#             #reset the substring index for next iteration with smaller encoded_text
#             substring_index = 0
        
#         substring_index += 1 
#     """
    
    

    return decoded_text

If you implemented the codebook, encode, and decode functions correctly, the code below should compress _War and Peace_ and return some sensible output.

In [5]:
with open("/data/warandpeace.txt", "r") as f:
    text = f.read()
    
character_counts = get_character_counts(text)
codebook = generate_codebook(character_counts)
encoded_text = encode(text, codebook)
decoded_text = decode(encoded_text, codebook)

print("First 1000 bits of encoded text: \n\n" +  encoded_text[:1000])
print("The total length of the encoded text is %d." % len(encoded_text))
print("First 1000 characters of decoded text:\n" + decoded_text[:1000])

First 1000 bits of encoded text: 

01110001110111100101110100101111010010010000100000000110100100101110000010000100011100110100101001110000100101001001011100001010111000010000010000010001110000001010000010000100000100111100000101110001010000111111010101100010100000010000010000010001110000000111000111101110001111000010000110001011100011110111000001011100001000001011100000010000100000111101110001110110000101110001010011100011101110100010000010000010000010000010001110000101000101100010100100110100101001110001001110000100000010000000010001011110001000001000000100110100101111110111101111101010001101001010000001011100100011011110101000111011000001000010000101111100110000110001011010010100000100001000001000000110001100001100110001011000000111001100110000100100010000100011000001111010101001010011011000011110110111110100000111111010101011001011111101001100001001100101011100111000100011100000000001110001001011010101101100000001011111101000101000101110000000000110101001000101111001010010011000000

When you are done, go to the terminal and type: 

`nbgrader submit Lab-03-29 --course dlsun`

This will submit your code so that it can be graded.